In [4]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pprint
import torch
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import math
plotly.tools.set_credentials_file(username='loneplay', api_key='pX8Z1nKtnRU4ppihpRrn')

In [5]:
gc_hps = [ 
          (400, 1.0, 3e-05),
           (400, 1.0, 1e-04),
           (600, 1.0, 3e-05),
           (800, 1.0, 1e-04),
           (1000, 1.0, 1e-04),
           (2500, 1.0, 1e-04),
           (2500, 2.0, 3e-05),
           (2500, 2.0, 3e-04),
           (5000, 1.0, 1e-04),
           (5000, 2.0, 3e-04),
           (10000, 1.0, 1e-04),
           (10000, 2.0, 3e-04),          
         ]

gc_paths = []

for cpt, alpha, lr in gc_hps:
    path = Path("/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k")
    for subdir in path.iterdir():
        ss = str(subdir)
        if "cpt{}_".format(cpt) in ss and \
           "msg{}x_".format(alpha) in ss and \
           "lr{:.0e}_".format(lr) in ss:
            gc_paths.append( ss )

pprint.pprint( gc_paths )


['/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr1e-04_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt600_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt800_nllmean_ce1.0_pg10.0_vf1.0_lr1e-04_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.47.joint_cpt1000_nllmean_ce1.0_pg10.0_vf1.0_lr1e-04_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/0

In [6]:
en_lm_files = []
for gc_path in gc_paths:
    for subdir in Path(gc_path).iterdir():
        ss = str(subdir)
        if "wikitext2_" in ss or \
           "coco_" in ss or \
           "multi30k_" in ss:
            en_lm_files.append( ss )

pprint.pprint(en_lm_files)

['/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/multi30k_Feedforward_lr5.0_drop0.5_filter4_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/multi30k_RNN_lr20.0_drop0.5_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/coco_RNN_lr10.0_drop0.5_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/coco_Feedforward_lr20.0_drop0.5_filter4_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf

In [9]:
#filters = ["multi30k_", "RNN"]
filters = ["multi30k_", "RNN", "20_"]

def satisfy(filters, string):
    for filter in filters:
        if not filter in string:
            return False
    return True

def lm_info(string):
    ss = ""
    lists = ["wikitext2_", "coco_", "multi30k_", "RNN", "Feedforward"]
    for ll in lists:
        if ll in string:
            ss += ll
    return ss

def model_info(string):
    cpt = string[string.find("joint_cpt") : string.find("_", string.find("joint_cpt")+7)]
    lr = string[string.find("lr") : string.find("_", string.find("lr"))]
    msg = string[string.find('msg') : string.find("msg")+6]
    return "{}_{}_{}".format(cpt, lr, msg)

lm_files = [xx for xx in en_lm_files if satisfy(filters, xx)]

print (len(lm_files))
pprint.pprint (lm_files)

12
['/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/multi30k_RNN_lr20_drop0.5_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt400_nllmean_ce1.0_pg10.0_vf1.0_lr1e-04_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/multi30k_RNN_lr20_drop0.5_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt600_nllmean_ce1.0_pg10.0_vf1.0_lr3e-05_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/multi30k_RNN_lr20_drop0.5_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.48.joint_cpt800_nllmean_ce1.0_pg10.0_vf1.0_lr1e-04_linear_ann100k_drop0.3_reinforce_msg1.0x_clip5.0_/multi30k_RNN_lr20_drop0.5_',
 '/checkpoint/jasonleeinf/groundcomms/decoding_180612_cpt_alpha/multi30k/06.12_13.47.joint_cpt1000_nllmean_ce1.0_pg10.0_vf1.0_lr1e-04_linear_ann100k_

In [11]:
traces = []
upto = 50000
for lm_file in lm_files:
    en_nll_lm = torch.load(lm_file)
    xx = list(en_nll_lm.keys())
    xx = [x for x in xx if x < upto]
    yy = [(en_nll_lm[k]) for k in xx]
    trace = go.Scatter(
        x = xx,
        y = yy,
        mode = 'lines',
        name = model_info(lm_file) + "_" +lm_info(lm_file)
    )
    traces.append(trace)

py.iplot(traces, filename = 'hi')


High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~loneplay/0 or inside your plot.ly account where it is named 'hi'
